In [1]:
from utils import *

# Preprocessing

Get the meme IDs, text and path to get the image caption

In [2]:
#EXIST 2024
exist_training_file = "../datasets/EXIST2021-2024_datasets/2024 EXIST/EXIST 2024 Memes Dataset/training/EXIST2024_training.json"
exist_test_file = "../datasets/EXIST2021-2024_datasets/2024 EXIST/EXIST 2024 Memes Dataset/test/EXIST2024_test_clean.json"

In [3]:
#MAMI
mami_training_file = "../datasets/MAMI DATASET/TRAINING/training.csv"
mami_test_file = "../datasets/MAMI DATASET/test/Test.csv"

In [4]:
data = {"dataset": {"EXIST2024": {}, "MAMI": {}}}

#EXIST
#training
exist_train_data_dict = preprocess_exist(exist_training_file)
data["dataset"]["EXIST2024"]["training"] = exist_train_data_dict

#test
exist_test_data_dict = preprocess_exist(exist_test_file,"test")
data["dataset"]["EXIST2024"]["test"] = exist_test_data_dict

#MAMI
#training
mami_train_data_dict = preprocess_mami(mami_training_file)
data["dataset"]["MAMI"]["training"] = mami_train_data_dict

#test
mami_test_data_dict = preprocess_mami(mami_test_file,"test")
data["dataset"]["MAMI"]["test"] = mami_test_data_dict


In [5]:
write_to_json(data,"datasets.json")

datasets.json is saved


# Post-processing after getting image captions

In [2]:
dataset_flan_coco = "datasets_with_captions_blip2-flan-coco_np.json"
captions_after_cleaning = "datasets_with_captions_after_cleaning.json"
captions_to_redo = "datasets_with_captions_to_redo_flan-coco.json"

In [3]:
removed_phrases = get_cleaned_data_and_instances_to_reprocess(dataset_flan_coco, dataset_flan_coco, captions_after_cleaning, captions_to_redo, n=3)

Saving data with cleaned image captions...
datasets_with_captions_after_cleaning.json is saved

Saving data to redo image captions...
datasets_with_captions_to_redo_flan-coco.json is saved


In [4]:
print_removed_phrases(removed_phrases)

Total captions that contained text from meme: 4021

Phrase: with the words 		 Total: 59
Phrase: with the caption 		 Total: 424
Phrase: with a caption 		 Total: 2422
Phrase: with the text 		 Total: 35
Phrase: with a text 		 Total: 47
Phrase: with a quote 		 Total: 121
Phrase: with a message 		 Total: 69
Phrase: with text saying 		 Total: 40
Phrase: with text that says 		 Total: 43
Phrase: and a caption 		 Total: 719
Phrase: and a sign that says 		 Total: 11
Phrase: and a quote that says 		 Total: 1
Phrase: and a text that says 		 Total: 1
Phrase: and a text saying 		 Total: 10
Phrase: and text that says 		 Total: 7
Phrase: and the caption says 		 Total: 1
Phrase: and saying 		 Total: 9
Phrase: with a funny meme 		 Total: 2


In [5]:
#how many memes are left to re-do image captioning
#empty set to store the meme IDs
redo_meme_ids = []

with open(captions_to_redo, "r", encoding="utf8") as fl:
        content = json.load(fl)

for dataset in content.values():
        for split in dataset.values():
            for memes in split.values():
                for meme_info in memes.values():
                    meme_text = meme_info["meme text"].lower()
                    meme_id = meme_info["meme id"]
                    redo_meme_ids.append(meme_id)

print(f"Memes to re-do image captioning: {len(redo_meme_ids)}")

Memes to re-do image captioning: 115


# After getting image captions, get representation for models

In [2]:
#After getting image captions:
data_with_captions = "datasets_with_captions_final.json"
#get text representation for svm and bert models
preprocessed_data = generate_text_caption_representations(data_with_captions)

In [3]:
#split the datasets into training and test for both MAMI and EXIST
process_and_save_splits(preprocessed_data)

Saved EXIST2024 training split to datasets/EXIST2024_training.json
Saved EXIST2024 test split to datasets/EXIST2024_test.json
Saved MAMI training split to datasets/MAMI_training.json
Saved MAMI test split to datasets/MAMI_test.json


# Include labels into datasets

In [2]:
datasets = ["MAMI", "EXIST2024"]
for dataset in datasets:
    add_labels(dataset, "training")
add_labels("MAMI", "test")

Saved MAMI training split to datasets/MAMI_training.json
Saved EXIST2024 training split to datasets/EXIST2024_training.json
Saved MAMI test split to datasets/MAMI_test.json


## Include meme paths into the final datasets! (10-04)

In [2]:
file_with_meme_paths = "datasets_with_captions_final.json"

mami_training_data = "../data/MAMI/MAMI_training.json"
mami_val_data = "../data/MAMI/MAMI_validation.json"
mami_test_data = "../preprocessing/datasets/MAMI_test.json"

exist_training_data = "../data/EXIST2024/EXIST2024_training.json"
exist_val_data = "../data/EXIST2024/EXIST2024_validation.json"
exist_test_data = "../data/EXIST2024/EXIST2024_test.json"

In [3]:
add_meme_path(file_with_meme_paths,mami_training_data)
add_meme_path(file_with_meme_paths,mami_val_data)
add_meme_path(file_with_meme_paths,mami_test_data)
add_meme_path(file_with_meme_paths,exist_training_data)
add_meme_path(file_with_meme_paths,exist_val_data)
add_meme_path(file_with_meme_paths,exist_test_data)

Saved ../data/MAMI/MAMI_training.json with image paths
Saved ../data/MAMI/MAMI_validation.json with image paths
Saved ../preprocessing/datasets/MAMI_test.json with image paths
Saved ../data/EXIST2024/EXIST2024_training.json with image paths
Saved ../data/EXIST2024/EXIST2024_validation.json with image paths
Saved ../data/EXIST2024/EXIST2024_test.json with image paths
